In [1]:
from dataManager.RainDataManager import RainDataManager
from dataManager.TempDataManager import TempDataManager
from dataManager.BikeDataManager import BikeDataManager
from datetime import datetime, timedelta, date
import matplotlib.dates as mdates
from matplotlib import pylab as plt
import seaborn as sns

In [2]:
rainDM = RainDataManager()
tempDM = TempDataManager()
bikeDM = BikeDataManager()

In [3]:
tempDM.loadData()
bikeDM.loadData()

In [4]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [12]:
start_date = date(2015, 1, 1)
end_date = datetime.now().date()
bikes = []
for single_date in daterange(start_date, end_date):
    print(str(single_date) + ' --> ' + str(bikeDM.getDayValue(single_date)))
    bikes.append(bikeDM.getDayValue(single_date))

2015-01-01 --> 140
2015-01-02 --> 429
2015-01-03 --> 373
2015-01-04 --> 260
2015-01-05 --> 500
2015-01-06 --> 227
2015-01-07 --> 715
2015-01-08 --> 639
2015-01-09 --> 681
2015-01-10 --> 443
2015-01-11 --> 323
2015-01-12 --> 788
2015-01-13 --> 848
2015-01-14 --> 957
2015-01-15 --> 764
2015-01-16 --> 456
2015-01-17 --> 383
2015-01-18 --> 265
2015-01-19 --> 409
2015-01-20 --> 197
2015-01-21 --> 276
2015-01-22 --> 318
2015-01-23 --> 612
2015-01-24 --> 370
2015-01-25 --> 286
2015-01-26 --> 730
2015-01-27 --> 576
2015-01-28 --> 472
2015-01-29 --> 487
2015-01-30 --> 97
2015-01-31 --> 133
2015-02-01 --> 71
2015-02-02 --> 487
2015-02-03 --> 497
2015-02-04 --> 453
2015-02-05 --> 479
2015-02-06 --> 320
2015-02-07 --> 376
2015-02-08 --> 229
2015-02-09 --> 570
2015-02-10 --> 701
2015-02-11 --> 670
2015-02-12 --> 694
2015-02-13 --> 521
2015-02-14 --> 120
2015-02-15 --> 144
2015-02-16 --> 169
2015-02-17 --> 325
2015-02-18 --> 604
2015-02-19 --> 651
2015-02-20 --> 382
2015-02-21 --> 200
2015-02-22 -->

KeyError: '4'

In [7]:
demand_loc = mdates.WeekdayLocator(3)
demand_fmt = mdates.DateFormatter('%a %b %d')

colors = sns.color_palette()
c1, c2, c3 = colors[0], colors[1], colors[2]

fig = plt.figure(figsize=(12, 6))

ax = fig.add_subplot(2, 1, 1)
ax.plot(daterange(start_date, end_date),
        bikes, lw=2, label="Full data")
ax.set_ylabel("Dayly demand")

NameError: name 'bikes' is not defined